In [186]:
# try run this if the following import doesn't work ! otherwise ask CharGPT! 
# commenting out the package you already have saves some time maybe......
!pip install numpy
!pip install pandas
!pip install geopandas
!pip install branca
!pip install folium
!pip install ipywidgets

Defaulting to user installation because normal site-packages is not writeable


In [22]:
import folium
import geopandas as gpd
import pandas as pd
import numpy as np
import ipywidgets as widgets
from ipywidgets import GridBox, Layout
from folium.features import GeoJsonTooltip
from folium.features import CustomIcon
from folium.plugins import MeasureControl
from IPython.display import display, clear_output, HTML
from branca.colormap import LinearColormap

## Set up + Load Data ##

In [32]:
# Path to shapefile + open
DA = r"C:\Users\zj1026.stu\OneDrive - UBC\FCOR599\Term1Presentation\Output_Data\GVan_DA_Final.shp"
gpdDA = gpd.read_file(DA)
# speed up calculation (from chatGPT): 
gpdDA = gpdDA.drop(columns=["DGUID","PRUID","LANDAREA"])
gpdDA["geometry"] = gpdDA.geometry.simplify(20, preserve_topology=True) # preserve_topology=True to avoid self-intersection issues.
# do this before reprojection otherwise the 20 will be in degree, not in meters

ID = "DAUID" # ID column

# Reprojection
if gpdDA.crs is not None and gpdDA.crs.to_epsg() != 4326:
    gpdDA = gpdDA.to_crs(epsg=4326)

gpdDA["centroid"] = gpdDA.geometry.centroid # do the centroid after the reprojection to get the correct position for centroid points, 
# because it's basically a calculated coordinate for the geometry that you already have. if you do this before reprojection, the geometry shift while the coordiates remains the same as before reprojection

# Define criteria 
criteria = ['CBDDis','StopDis','popdensity','H_maintain','H_rent','green_dens','H_edu','Age_','Unemploy_R','MedianInc','DIST_Coast','dis_scho','dis_medi','SurfaceT','PM2_5_Mean','CrimeRate']
cost_criteria = ['StopDis','H_maintain','H_rent','Unemploy_R','dis_scho','dis_medi','SurfaceT','PM2_5_Mean','CrimeRate']  # currently all are "the smaller, the better"
benefit_criteria = ['green_dens','H_edu'] 
unknown_criteria = ['CBDDis','popdensity','Age_','MedianInc','DIST_Coast']

C:\Users\zj1026.stu\AppData\Local\Temp\ipykernel_25092\1829373680.py:15: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gpdDA["centroid"] = gpdDA.geometry.centroid # do the centroid after the reprojection to get the correct position for centroid points,


In [33]:
gpdDA

,DAUID,StopDis,CBDDis,H_maintain,H_rent,popdensity,green_dens,H_edu,Age_,Unemploy_R,...,DIST_Coast,dis_scho,dis_medi,SurfaceT,PM2_5_Mean,CrimeRate,Shape_Leng,Shape_Area,geometry,centroid
0,59150004,299.15,8920.30,2950.47,2860.02,425.0,0.12,150,52,6,...,2.99,12956.48,4629.36,25.65,3.83,2.57e-03,11483.58,8.51e+05,"POLYGON ((-123.27497 49.36885, -123.27552 49.3...",POINT (-123.25834 49.38879)
1,59150005,76.09,8742.44,2320.00,1920.00,3168.7,0.11,145,43,7,...,425.38,11200.47,3181.21,30.45,3.83,1.32e-02,2038.22,1.66e+05,"POLYGON ((-123.27552 49.36948, -123.27603 49.3...",POINT (-123.27675 49.37161)
2,59150006,135.77,8984.06,2680.00,1280.00,2899.3,0.14,195,44,12,...,324.34,11333.97,3389.08,29.87,3.83,1.40e-02,2495.17,1.56e+05,"POLYGON ((-123.27673 49.37536, -123.2776 49.37...",POINT (-123.28009 49.37237)
3,59150007,509.44,9373.56,2701.37,1771.88,755.8,0.12,240,49,7,...,220.31,11665.32,3770.47,26.81,3.83,3.03e-03,6335.65,7.21e+05,"MULTIPOLYGON (((-123.29247 49.36949, -123.2927...",POINT (-123.28367 49.37494)
4,59150008,373.90,8234.88,3046.36,2619.72,903.0,0.11,275,46,0,...,316.39,9849.44,2148.98,27.66,3.83,3.31e-03,12275.26,6.59e+05,"MULTIPOLYGON (((-123.27642 49.35336, -123.2770...",POINT (-123.27685 49.35921)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3586,59154402,102.02,0.00,1300.00,1360.00,2283.4,0.02,15,60,0,...,4999.88,3554.52,99.80,36.17,3.73,1.92e-02,1729.33,1.49e+05,"POLYGON ((-122.77529 49.2552, -122.77855 49.25...",POINT (-122.77694 49.25803)
3587,59154404,235.01,1628.23,1360.00,940.00,204.2,0.02,40,43,11,...,6776.15,4126.67,1361.05,34.51,3.73,8.88e-04,11095.24,3.22e+06,"POLYGON ((-122.77943 49.26455, -122.73382 49.2...",POINT (-122.75685 49.24899)
3588,59154406,272.17,5891.70,2140.00,2200.00,974.5,0.13,295,42,7,...,940.54,4130.24,3611.78,29.01,3.83,3.40e-03,3982.20,8.09e+05,"POLYGON ((-122.96783 49.31601, -122.95554 49.3...",POINT (-122.96301 49.31171)
3589,59154407,285.93,4946.32,2155.82,1782.38,2123.3,0.11,870,42,8,...,248.73,3075.93,3328.46,28.69,3.83,2.46e-03,5092.12,1.12e+06,"POLYGON ((-122.9912 49.31287, -122.96894 49.31...",POINT (-122.97872 49.3094)


## Slider + Weight Control ##

In [34]:
# Sliders: 0–10 where 0 = doesn't care, 10 = very important
label = ['Distance to CBD (m)', 'Distance to Transit Stations (m)', 'Population Density','Owned House Maintainance fee ($)',
         'House Rent ($)','Green Density','Percentage of Bachelor Degree Holders','Age','Unemployment Rate', 'Median Income ($)',
         'Distance to Seashore (m)', 'Distance to Nearest School (m)', 'Distance to Hospital (m)', 'Surface Temperatrue (°C)', 
         'PM2.5 Concentration (μg/m³)','Crime Rate (/m²)']
# update mandatory!!!!!!

sliders = {c: widgets.IntSlider(value=1,  # default equal importance. c: is the key
                                min=0,
                                max=10,
                                step=1,
                                description=l, #slider label, not the dic key
                                continuous_update=False,
                                style={"description_width": "250px"},
                                layout=widgets.Layout(width="600px",border="2px solid #0ABAB5",padding="5px",margin="5px")) for c,l in zip(criteria,label)} # output:dictionary
slider_format = GridBox(list(sliders.values()), layout=Layout(grid_template_columns="repeat(2, 600px)", # 2 columns, 600px each cell
                                                             grid_gap="0px 20px", #row gap, col gap
                                                             justify_items="center"))


# toggle options:
label_unknown = ['Distance to CBD:', 'Population Density:','Average Age:','Median Income:',
                 'Distance to Seashore']
# update mandatory!!!!!!
toggles = {f"{c}_pref": widgets.ToggleButtons(options=[('Lower','lower'),('Higher','higher')],
                                       value = 'lower',
                                       description=l,
                                       style={"description_width": "50px"},
                                       layout=widgets.Layout(width="180px",border="2px solid #0ABAB5",padding="5px",margin="5px")) for c,l in zip(unknown_criteria,label_unknown)}


# arrange the sliders and the toggles
sliders_box = widgets.VBox([widgets.HTML("<h3>Set Preference for Each Criterion (0–10)</h3>"),
                            slider_format,
                            widgets.HTML("<h3>Decide Your Preference:</h3>"),
                            widgets.HBox(list(toggles.values()))]) # list() to make it save, sometimes Hbox doesn't take dictionary as an input, but only iterable object like list.
                            # put widgets.Hbox() inside to make sure there is only one toggle frame and it's list in: Vertical Vertical ... last item->horizontal 
output = widgets.Output()

In [39]:
# preview the slider bar
# display(sliders_box)

## Normalize Criteria & Compute Suitability ##

In [36]:
def compute_suitability(df, weight, preference_value):
    df = df.copy()

    # Normalize weights to 0–100% 
    totalweight = sum(weight.values()) # the result of dragging slider bar is also a dictionary
    if totalweight == 0:
        n = len(weight)
        weight_norm = {c: 1 / n for c in weight} # if 0 for all, then use default where each criterion get the same weight
    else:
        weight_norm = {c: v / totalweight for c, v in weight.items()}
    weight_percent = {c: 100 * w for c, w in weight_norm.items()}


    # Normalize each criterion to 0-1
    for col in criteria:
        vals = df[col].astype(float) # get the entire column from the df, based on the name of col
        vmin, vmax = vals.min(), vals.max() # get the extent of the column

        if vmax == vmin:
            df[col + "_norm"] = 0.0 # fill 0 to all rows if the extent is 0 as well. 
            continue

        # decide if CBD is cost/benefit
        if col in unknown_criteria:
            user_pref = preference_value[col]
            if user_pref == 'lower':
                df[col + "_norm"] = (vmax - vals) / (vmax - vmin)
            else:
                df[col + "_norm"] = (vals - vmin) / (vmax - vmin)

        # for other normal criteria
        elif col in cost_criteria:
            # if the criterion is better when the number is small (cost)
            df[col + "_norm"] = (vmax - vals) / (vmax - vmin)
        else:
            # if the criterion is better when the number is large (benifit)
            df[col + "_norm"] = (vals - vmin) / (vmax - vmin)

    # Calculate suitability score
    df["suit_score"] = sum(df[c + "_norm"] * weight_norm[c] for c in criteria)

    return df, weight_norm, weight_percent

## Build Interactive Map Function ##

In [37]:
# get the map center
center_y = gpdDA.geometry.centroid.y.mean()
center_x = gpdDA.geometry.centroid.x.mean()
# Hover tooltip: ID, score, and raw criterion values
tooltip_fields = [ID, "suit_score"] + criteria
tooltip_aliases = [ID, "Suitability Score"] + label 

def update_map(**all_inputs): # accept any length of the input as keyword arguments( a=1; b=2; c....), then transform it into a dictionary, so weight will be interpretate as an dictionary
    with output: # printed or displayed inside this block goes into the Output widget, so everything below is incorporated into the widget window, not a jupyter cell.
        clear_output() # clear everything before (each movement of the slider actually produce one map, so clear it before get the final map), also see rational below

        weight = {c: all_inputs[c] for c in criteria}
        preference_value = {c: all_inputs[f"{c}_pref"] for c in unknown_criteria}

        # call function above to get the all necessary results:
        df_score, weight_norm, weight_percent = compute_suitability(gpdDA, weight, preference_value)

        #rank + slcie the top 10 most promising living site 
        sorted_score = df_score.sort_values("suit_score", ascending=False)
        top10 = sorted_score.head(10).copy()
        
        top10["Rank"] = np.arange(1, len(top10) + 1)      
        # get the summary table for top 10
        cols_for_table = ["Rank", ID, "suit_score"] + criteria
        summary_table = top10[cols_for_table].copy()
        summary_table = summary_table.sort_values("Rank")


        # print normalized weights just for reference
        print("Normalized weights (auto-scaled to sum to 100%):")
        nw = '|'.join([f"  {c}: {weight_percent[c]:.1f}%" for c in criteria])
        print(nw)
            # 10 & 5 = width; .1 = preserve 1 decimal after the point; s = string; f = float, c: & weight_percent[c]: = define the format of this item for me 
        
        # Basemap
        m = folium.Map(location=[center_y, center_x],
                       zoom_start=10.5,
                       tiles='https://{s}.basemaps.cartocdn.com/rastertiles/voyager/{z}/{x}/{y}{r}.png', 
                       # https://leaflet-extras.github.io/leaflet-providers/preview/
                       # all possible tiles to choose 
                       attr='CartoDB.Voyager', 
                       max_zoom=20)
        # measuring tool
        MeasureControl(position='bottomleft',
                       primary_length_unit='meters',
                       secondary_length_unit='kilometers').add_to(m)
        
        # Choropleth based on suitability score
        linear = LinearColormap(["#4281a4", "#9cafb7", "#ead2ac", "#e6b89c", "#fe938c"], 
                                vmin=df_score["suit_score"].min(), 
                                vmax=df_score["suit_score"].max(), 
                                caption = "Suitability Score").add_to(m)
        # modify the choropleth map + adding tooltip
        df_for_map = df_score.drop(columns=["centroid"])
        folium.GeoJson(
            df_for_map,
            name="Info",
            style_function=lambda feature: {"color": "black", # outline/border color
                                            "fillColor": linear(feature['properties']['suit_score']), # GeoJSON dataframe = feature["properties"] + feature["geometry"]
                                            "weight": 0.3,
                                            "fillOpacity": 0.8},
            tooltip=GeoJsonTooltip(fields=tooltip_fields,
                                   aliases=tooltip_aliases,
                                   localize=True, # format the numbers well: adds commas + format float + respects locale settings (round up is no space)
                                   sticky=False)).add_to(m)  # disapper if cursor move off the polygon

        
        # Mark top 10 DA centroids in red
        for _, row in top10.iterrows(): # .iterrows()loop through a pandas DataFrame row by row. return index + row content 
            #_ = i dont care, so here means I don't care the index.
            c = row.centroid
            popup = f"""
            <div style=" font-size:14px; padding:8px; background-color:#ffe8c1; border-radius:8px; border: 2px solid #555">
            <b> Most Suitable Place to Live Number #{int(row['Rank'])}</b><br>
            <b>{ID}</b>: {row[ID]}<br>
            <b>Suitability Score</b>: {row['suit_score']:.3f}<br>
            <br>
            <b>Distance to CBD (m)</b>: {row['CBDDis']}<br>
            <b>Distance to Public Transportation (m)</b>: {row['StopDis']}<br>
            <b>Population Density</b>: {row['popdensity']}<br>
            <b>Owned House Maintainance fee ($)</b>: {row['H_maintain']}<br>
            <b>House Rent ($)</b>: {row['H_rent']}<br>
            <b>Green Density</b>: {row['green_dens']}<br>
            <b>Percentage of Bachelor Degree Holders</b>: {row['H_edu']}<br>
            <b>Age</b>: {row['Age_']}<br>
            <b>Unemployment Rate</b>: {row['Unemploy_R']}<br>
            <b>Median Income</b>: {row['MedianInc']}<br>
            <b>Distance to Seashore</b>: {row['DIST_Coast']}<br>
            <b>Distance to Nearest School</b>: {row['dis_scho']}<br>
            <b>Distance to Hospital</b>: {row['dis_medi']}<br>
            <b>Surface Temperatrue (°C)</b>: {row['SurfaceT']}<br>
            <b>PM2.5 Concentration (μg/m³)</b>: {row['PM2_5_Mean']}<br>
            <b>Crime Rate (/m²)</b>: {row['CrimeRate']}<br>
            </div>
            """
            popup_text = folium.Popup(popup, max_width = '400')

            folium.Marker(location=[c.y, c.x],
                          popup=popup_text,
                          icon=folium.DivIcon(
                              html="""
                              <div style="font-size:20px; 
                              line-height:40px; 
                              transform: translate(-50%, -50%);">🏵️</div>
                              """)).add_to(m)
            # text-shadow: 0 0 5px #ead2ac, 0 0 8px #9cafb7, 0 0 15px #4281a4;  for shadow, 008pxgold -> 00 shift in x&y, size, color
            # line-height: control the invisible sqaure that contain the emoji, transform: translate(-50%, -50%), put the image at centre, default is top-left corner
        
        # print map
        display(m)
        
        print("\nTop 10 most suitable dissemination areas:")
        summary_table.columns = (["Rank", "DAUID", "Suitability Score"] + label)  # existing list of 5 nicely formatted names
        # set the dataframe format
        pd.set_option("display.precision", 2)
        pd.set_option("display.width", 2000)
        display(summary_table)

C:\Users\zj1026.stu\AppData\Local\Temp\ipykernel_25092\1173824240.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center_y = gpdDA.geometry.centroid.y.mean()
C:\Users\zj1026.stu\AppData\Local\Temp\ipykernel_25092\1173824240.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center_x = gpdDA.geometry.centroid.x.mean()


## Connect Map & Control Panel <br> + Display the Map ##

In [38]:
interactive = widgets.interactive_output(update_map, {**sliders, **toggles}) # build connection between the slider bar and the map, so that's how the change in the slider actually affect the map display
# however, that means everytime we adjust the slider, a new set of values are put into the function, so the whole function will run again. that's why we need a clear_output() above.
# widgets.interactive_output(function, controls) where controls MUST be a DICTIONARY, the function will know how to match the parameters by the KEYs in the control dictionary. 
display(sliders_box, output) # show slider_box and the output(map + table) in sequence 

Output()